Image resizing
https://blog.roboflow.com/you-might-be-resizing-your-images-incorrectly/

We have two set of images
Mike : 3712x2088
Neil : 5120x3840

We need to resize
1. resize the image
2. crop to be square

recreate folder structure and resize

In [8]:
import matplotlib.pyplot as plt
import skimage.transform
import scipy.misc 
import imageio
import math
import numpy as np
import os

In [9]:
IMG_WIDTH = 255
MAKE_SQUARE = True

#read paths
MIKE_PROCESSED = r'C:\DS Project\Images\Mike\Processed' # all other photos after tractors selected
NEIL_PROCESSED = r'C:\DS Project\Images\Neil\Processed' # all other photos after tractors selected

MIKE_TRACTORS = r'C:\DS Project\Images\classified_Tractors\Mike' # selected and hand classified tractors
NEIL_TRACTORS = r'C:\DS Project\Images\classified_Tractors\Neil' # selected and hand classified tractors

#save paths
MIKE_PROCESSED_RESIZED = r'C:\DS Project\_resized\Images\resized_'+ str(IMG_WIDTH) + ('_sq' if MAKE_SQUARE == True else '_') + r'\processed\Mike'
NEIL_PROCESSED_RESIZED = r'C:\DS Project\_resized\Images\resized_'+ str(IMG_WIDTH) + ('_sq' if MAKE_SQUARE == True else '_') + r'\processed\Neil'

TRACTORS_MIKE_TO_RESIZED = r'C:\DS Project\_resized\Images\resized_'+ str(IMG_WIDTH) + ('_sq' if MAKE_SQUARE == True else '_') + r'\tractors\Mike\to'
TRACTORS_NEIL_TO_RESIZED = r'C:\DS Project\_resized\Images\resized_'+ str(IMG_WIDTH) + ('_sq' if MAKE_SQUARE == True else '_') + r'\tractors\Neil\to'

TRACTORS_MIKE_FROM_RESIZED = r'C:\DS Project\_resized\Images\resized_'+ str(IMG_WIDTH) + ('_sq' if MAKE_SQUARE == True else '_') + r'\tractors\Mike\from'
TRACTORS_NEIL_FROM_RESIZED = r'C:\DS Project\_resized\Images\resized_'+ str(IMG_WIDTH) + ('_sq' if MAKE_SQUARE == True else '_') + r'\tractors\Neil\from'
        

In [10]:
def bound_image_dim(image, min_size=None, max_size=None):
    
    if (max_size is not None) and \
       (min_size is not None) and \
       (max_size < min_size):
        raise ValueError('`max_size` must be >= to `min_size`')
        
    dtype = image.dtype
    (height, width, *_) = image.shape
    
    # scale the same for both height and width for fixed aspect ratio resize
    scale = 1
    
    # bound the smallest dimension to the min_size
    if min_size is not None:
        image_min = min(height, width)
        scale = max(1, min_size / image_min)
    
    # next, bound the largest dimension to the max_size
    # this must be done after bounding to the min_size
    if max_size is not None:
        image_max = max(height, width)
        if round(image_max * scale) > max_size:
            scale = max_size / image_max
    if scale != 1:
        image = skimage.transform.resize(
            image, (round(height * scale), round(width * scale)),
            order=1,
            mode='constant',
            preserve_range=True)
    return image.astype(dtype)


def square_pad_image(image, size):
    
    #get dimensions
    (height, width, *_) = image.shape
    
    # check 
    if (size < height) or (size < width):
        raise ValueError('`size` must be >= to image height and image width')
        
    pad_height = (size - height) / 2
    pad_top = math.floor(pad_height)
    pad_bot = math.ceil(pad_height)
    pad_width = (size - width) / 2
    pad_left = math.floor(pad_width)
    pad_right = math.ceil(pad_width)
    return np.pad(
        image, ((pad_top, pad_bot), (pad_left, pad_right), (0, 0)),
        mode='constant')


def createFolders():
    
    try:
        os.makedirs(MIKE_PROCESSED_RESIZED)
    except OSError:
        print ("Creation of the directory %s failed" % MIKE_PROCESSED_RESIZED)
    else:
        print ("Successfully created the directory %s " % MIKE_PROCESSED_RESIZED)

    try:
        os.makedirs(NEIL_PROCESSED_RESIZED)
    except OSError:
        print ("Creation of the directory %s failed" % NEIL_PROCESSED_RESIZED)
    else:
        print ("Successfully created the directory %s " % NEIL_PROCESSED_RESIZED)

    try:
        os.makedirs(TRACTORS_MIKE_TO_RESIZED)
    except OSError:
        print ("Creation of the directory %s failed" % TRACTORS_MIKE_TO_RESIZED)
    else:
        print ("Successfully created the directory %s " % TRACTORS_MIKE_TO_RESIZED)

    try:
        os.makedirs(TRACTORS_NEIL_TO_RESIZED)
    except OSError:
        print ("Creation of the directory %s failed" % TRACTORS_NEIL_TO_RESIZED)
    else:
        print ("Successfully created the directory %s " % TRACTORS_NEIL_TO_RESIZED)
  
    try:
        os.makedirs(TRACTORS_MIKE_FROM_RESIZED)
    except OSError:
        print ("Creation of the directory %s failed" % TRACTORS_MIKE_FROM_RESIZED)
    else:
        print ("Successfully created the directory %s " % TRACTORS_MIKE_FROM_RESIZED)

    try:
        os.makedirs(TRACTORS_NEIL_FROM_RESIZED)
    except OSError:
        print ("Creation of the directory %s failed" % TRACTORS_NEIL_FROM_RESIZED)
    else:
        print ("Successfully created the directory %s " % TRACTORS_NEIL_FROM_RESIZED)
    
    
def resizeProcessedImages(readPath, savePath): 

    print ("Resizing images in ",readPath)

    # loop round each image in the original folder
    for subdir, dirs, files in os.walk(readPath):
        
        for filename in files:
        
            filepath = subdir + os.sep + filename
            origImg = imageio.imread(filepath)

            h,w,c = origImg.shape
            
            resizedImg = bound_image_dim(origImg,0,IMG_WIDTH)
            
            if (MAKE_SQUARE == True):
                    squaredImg = square_pad_image(resizedImg, IMG_WIDTH)
                    plt.imsave(savePath + r'\\' + filename, squaredImg)
                
            else:
                    plt.imsave(savePath + r'\\' + filename, resizedImg)
    
def resizeClassifiedImages(Path, toFilePath_To, toFilePath_From):
      
    toImages = Path + r'\to'
    fromImages = Path + r'\from'
    
    #loop round each 'to' images, resixe and save on 'to folder
    for filename in os.listdir(toImages):
       
        filepath = toImages + os.sep + filename
        origImg = imageio.imread(filepath)

        h,w,c = origImg.shape
    
        resizedImg = bound_image_dim(origImg,0,IMG_WIDTH)
            
        if (MAKE_SQUARE == True):
                squaredImg = square_pad_image(resizedImg, IMG_WIDTH)
                plt.imsave(toFilePath_To + r'\\' + filename, squaredImg)
                
        else:
                plt.imsave(toFilePath_To + r'\\' + filename, resizedImg)
            
        print ("processed..", toFilePath_To + r'\\' + filename)
    
    #loop round each 'from' image, resize and save in 'from' folder
    for filename in os.listdir(fromImages):
       
        filepath = fromImages + os.sep + filename
        origImg = imageio.imread(filepath)

        h,w,c = origImg.shape
    
        resizedImg = bound_image_dim(origImg,0,IMG_WIDTH)
            
        if (MAKE_SQUARE == True):
                squaredImg = square_pad_image(resizedImg, IMG_WIDTH)
                plt.imsave(toFilePath_From + r'\\' + filename, squaredImg)
                
        else:
                plt.imsave(toFilePath_From + r'\\' + filename, resizedImg)
            
        print ("processed..", toFilePath_From + r'\\' + filename)
     
  
         

In [11]:
#create folders
createFolders()
print("folders created")

#resize mike processed
resizeProcessedImages(MIKE_PROCESSED,MIKE_PROCESSED_RESIZED)
print("resizeed mike processed")

#resize neil processed
resizeProcessedImages(NEIL_PROCESSED,NEIL_PROCESSED_RESIZED)
print("processed neil processed")

#process mike tractors
#resizeClassifiedImages(MIKE_TRACTORS, TRACTORS_MIKE_TO_RESIZED, TRACTORS_MIKE_FROM_RESIZED)
print("processed mike tractors")

#process neil tractors
#resizeClassifiedImages(NEIL_TRACTORS, TRACTORS_NEIL_TO_RESIZED, TRACTORS_NEIL_FROM_RESIZED)
print("processed neil tractors")

print("all images resized")



Creation of the directory C:\DS Project\_resized\Images\resized_255_sq\processed\Mike failed
Successfully created the directory C:\DS Project\_resized\Images\resized_255_sq\processed\Neil 
Creation of the directory C:\DS Project\_resized\Images\resized_255_sq\tractors\Mike\to failed
Creation of the directory C:\DS Project\_resized\Images\resized_255_sq\tractors\Neil\to failed
Creation of the directory C:\DS Project\_resized\Images\resized_255_sq\tractors\Mike\from failed
Creation of the directory C:\DS Project\_resized\Images\resized_255_sq\tractors\Neil\from failed
folders created
Resizing images in  C:\DS Project\Images\Mike\Processed
resizeed mike processed
Resizing images in  C:\DS Project\Images\Neil\Processed
processed neil processed
processed mike tractors
processed neil tractors
all images resized
